[SQL HAVING Clause](https://www.w3schools.com/sql/sql_having.asp)

<b> Syntaxe : </b><br>
SELECT column_name(s) FROM table_name
WHERE condition <br>
GROUP BY column_name(s) <br>
HAVING condition <br>
ORDER BY column_name(s);

<b> Objet de la requête : </b><br>
La clause HAVING a été ajoutée à SQL parce que le mot clé WHERE ne peut pas être utilisé avec les fonctions d'agrégation.

Date : 26-01-24

In [1]:
import duckdb
import polars as pl
from skimpy import skim

In [2]:
# Création d'une BD virtuelle
conn = duckdb.connect()

In [3]:
# Requête SQL : récupération de toutes les données du fichier .csv dans un DF
df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/prisoners.csv', header=True)
    """
).df()

In [5]:
# Stats de la DF
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 553    │ │ string      │ 9     │                                                          │
│ │ Number of columns │ 18     │ │ datetime64  │ 4     │                                                          │
│ └───────────────────┴────────┘ │ int32       │ 3     │                                                          │
│                                │ float64     │ 2     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name            ┃ NA  ┃ NA %  ┃ mean    ┃ sd      ┃ p0   ┃ p25  ┃ p50    ┃ p75    ┃ p100   ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━┩  │
│ │ Execution              │   0 │     0 │     277 │   159.8 │    1 │  139 │    277 │    415 │    553 │ ▇▇▇▇▇▇ │  │
│ │ Highest Education      │  48 │  8.68 │   10.21 │   2.096 │    3 │    9 │     10 │     12 │     20 │   ▃▇▅  │  │
│ │ Level                  │     │       │         │         │      │      │        │        │        │        │  │
│ │ TDCJ                   │   0 │     0 │  524400 │  499100 │  511 │  815 │ 999000 │ 999300 │ 999600 │ ▇    ▇ │  │
│ │ Number                 │     │       │         │         │      │      │        │        │        │        │  │
│ │ Age at Execution       │   0 │     0 │   39.47 │   8.628 │   24 │   33 │     38 │     44 │     67 │ ▅▇▇▃▂▁ │  │
│ │ Weight                 │  42 │  7.59 │     176 │    32.9 │  104 │  151 │    170 │  191.5 │    318 │  ▁▇▅▂  │  │
│ └────────────────────────┴─────┴───────┴─────────┴─────────┴──────┴──────┴────────┴────────┴────────┴────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                 ┃ NA    ┃ NA %     ┃ first              ┃ last               ┃ frequency       ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩  │
│ │ Date of Birth               │    14 │     2.53 │     1931-07-31     │     1997-02-15     │ None            │  │
│ │ Date of Offence             │    17 │     3.07 │     1974-03-19     │     2009-03-11     │ None            │  │
│ │ Date Received               │    14 │     2.53 │     1974-08-26     │     2010-03-16     │ None            │  │
│ │ Execution Date              │     0 │        0 │     1982-12-07     │     2018-07-17     │ None            │  │
│ └─────────────────────────────┴───────┴──────────┴────────────────────┴────────────────────┴─────────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                  ┃ NA       ┃ NA %        ┃ words per row              ┃ total words           ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━

In [4]:
# Enregistrement de la DF ci-avant dans une BD virtuelle (équivalent au CTE)
conn.register('df_view', df)

Requête SQL : Affichage des 5 premières lignes du nombre de prisonniers > à 10 par ville par trie décroissant

In [6]:
conn.execute(
    """
    -- Champs à récupérer
    SELECT 
        "County" AS "Ville"
        , COUNT("Execution") AS "Nombre de prisonniers"
    -- Depuis la BD virtuelle (CTE)
    FROM df_view
    -- TCD nombre de prisonniers par ville
    GROUP BY "Ville"
    -- Filtre TCD : ville avec + de 10 prisonniers
    HAVING "Nombre de prisonniers" > 10
    -- Trie décroissant
    ORDER BY "Nombre de prisonniers" DESC
    -- Affichage des 5 premières lignes
    LIMIT 5
    """
).df()

,Ville,Nombre de prisonniers
0,Harris,128
1,Dallas,58
2,Bexar,46
3,Tarrant,41
4,Nueces,16
